<a href="https://colab.research.google.com/github/Mabzak-Knight/manga-web-downloader/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
#@title Instalisasi
#@markdown Paket program
!pip install requests
!pip install beautifulsoup4
!pip install fpdf
!pip install reportlab

import requests
from bs4 import BeautifulSoup
import os

def check_image_size(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Memeriksa panjang konten (size) dari respons
        content_length = int(response.headers['Content-Length'])
        if content_length > 0:
            return True
        else:
            return False
    else:
        return False

def get_image_urls_section(url, class_name, tag_name):
    """
    Mengambil URL dari semua gambar di dalam elemen <div> dengan kelas tertentu.

    Args:
        url (str): URL dari website.
        class_name (str): Nama kelas dari elemen <div> yang mengandung gambar.

    Returns:
        list: Daftar URL dari gambar.
    """
    # Mengirim permintaan GET untuk mengunduh halaman web
    response = requests.get(url)

    # Memeriksa apakah permintaan berhasil
    if response.status_code == 200:
        # Menggunakan BeautifulSoup untuk mengurai HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mencari elemen <section> dengan kelas yang diberikan
        div_element = soup.find(tag_name, {'id': class_name})

        # Jika elemen ditemukan, cari semua elemen gambar di dalamnya
        if div_element:
            images = div_element.find_all('img')

            # Mengambil URL dari setiap gambar
            image_urls = [image['src'] for image in images]
            return image_urls
        else:
            print(f"Elemen <div> dengan kelas '{class_name}' tidak ditemukan.")
    else:
        print(f"Gagal mengakses {url}. Status code: {response.status_code}")


def download_images(image_urls, folder_path):
    for i, image_url in enumerate(image_urls):
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(os.path.join(folder_path, f'image_{i}.jpg'), 'wb') as f:
                f.write(response.content)
            print(f'Downloaded image_{i}.jpg')
        else:
            print(f'Failed to download image_{i}.jpg')

def download_imageurl(url, folder_path, index):
    response = requests.get(url)
    if response.status_code == 200:
        if check_image_size(url):
          # Mengonversi indeks ke format string dengan padding nol
          index_str = str(index).zfill(3)

          # Membuat nama file dengan format "image_001.jpg"
          file_name = f'image_{index_str}.jpg'

          # Menggabungkan path folder dengan nama file
          file_path = os.path.join(folder_path, file_name)

          # Menyimpan konten gambar ke file
          with open(file_path, 'wb') as f:
              f.write(response.content)
              print(f'Gambar berhasil diunduh dan disimpan di {file_path}')
        else:
            print(f"Gambar Memiliki ukuran nol dari URL: {url}")
    else:
        print(f'Gagal mengunduh gambar dari URL: {url}')


In [53]:
#@title Cek judul
import requests
from bs4 import BeautifulSoup

# URL dari website yang ingin Anda baca judulnya
url = "https://komiku.id/ch/httpsadmin-komiku-orgmajutsushi-kunon-wa-miete-iru-chapter-07-2/"  # @param {type:"string"}

# Mengirim permintaan GET untuk mengunduh halaman web
response = requests.get(url)

# Memeriksa apakah permintaan berhasil
if response.status_code == 200:
    # Menggunakan BeautifulSoup untuk mengurai HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Mencari elemen judul
    title_element = soup.find('title')

    # Memeriksa apakah elemen judul ditemukan
    if title_element is not None:
        # Mendapatkan teks dari elemen judul
        title = title_element.get_text()
        print(f"Judul dari {url} adalah: {title}")
    else:
        print(f"Tidak dapat menemukan elemen judul di {url}")
else:
    print(f"Gagal mengakses {url}. Status code: {response.status_code}")


Judul dari https://komiku.id/ch/httpsadmin-komiku-orgmajutsushi-kunon-wa-miete-iru-chapter-07-2/ adalah: Komik Majutsushi Kunon wa Miete Iru Chapter 07.2 - Komiku


In [140]:
#@title Download gambar pada website
# Contoh penggunaan:
url = "https://komiku.id/ch/httpsadmin-komiku-orgmajutsushi-kunon-wa-miete-iru-chapter-07-1/#2"   # @param {type:"string"}
id_name = "Baca_Komik"  # @param {type:"string"}
tag_name = "section"  # @param {type:"string"}

folder_path = "./downloaded_image11" # @param {type:"string"}
os.makedirs(folder_path, exist_ok=True)

image_urls = get_image_urls_section(url, id_name, tag_name)

if image_urls:
    index = 1  # Initialize index
    for url in image_urls:
        # Memanggil fungsi untuk mendownload dan menyimpan gambar
        url = quote(url, safe='/:')
        download_imageurl(url, folder_path, index)
        index += 1  # Increment index for the next image

In [74]:
#@title Zip folder
import zipfile
import os

def zip_folder(folder_path, zip_name):
    # Membuat objek zip
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Melakukan iterasi di seluruh file dalam folder dan menambahkannya ke dalam zip
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# Contoh penggunaan
folder_topath = folder_path #@param
zip_name = 'output.zip' #@param {type:"string"}

zip_folder(folder_topath, zip_name)


In [ ]:
#@title Pdf Folder
import img2pdf
import os

def images_to_pdf(image_folder, output_pdf):
    # Membuat daftar path file gambar
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # Mengonversi gambar ke PDF
    with open(output_pdf, "wb") as f:
        f.write(img2pdf.convert(image_files))

# Contoh penggunaan
folder_topath = folder_path #@param
pdf_name = 'output.pdf' #@param {type:"string"}

images_to_pdf(folder_topath, pdf_name)